<a href="https://colab.research.google.com/github/Chiaradisanto/Segmentation/blob/main/sensorfunzionante5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [26]:
import os
batch_size=3

In [27]:
seed=24
batch_size= 3
n_classes=1
from tensorflow.keras.preprocessing.image import ImageDataGenerator




#Define the generator.
#We are not doing any rotation or zoom to make sure mask values are not interpolated.
#It is important to keep pixel values in mask as 0, 1, 2, 3, .....
def trainGenerator(train_img_path, train_mask_path):
    
    img_data_gen_args = dict(horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect',rescale=1/255.)
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)
    mask_datagen = ImageDataGenerator(**img_data_gen_args)
   
    
    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        class_mode = None,
        color_mode = 'grayscale',
        target_size=(512,512),
        batch_size = batch_size,
        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_mask_path,
        class_mode = None ,
        color_mode = 'grayscale',
        target_size=(512,512),
        batch_size = batch_size,
        seed = seed)
    
    train_generator = zip(image_generator, mask_generator)
    return train_generator

In [28]:
train_img_path = "/gdrive/MyDrive/TESI/prova/data/train_images"
train_mask_path = "/gdrive/MyDrive/TESI/prova/data/train_masks"
train_img_gen = trainGenerator(train_img_path, train_mask_path)

val_img_path = "/gdrive/MyDrive/TESI/prova/data/val_images"
val_mask_path = "/gdrive/MyDrive/TESI/prova/data/val_masks"
val_img_gen = trainGenerator(val_img_path, val_mask_path)

Found 384 images belonging to 1 classes.
Found 384 images belonging to 1 classes.
Found 96 images belonging to 1 classes.
Found 96 images belonging to 1 classes.


In [29]:
x, y = train_img_gen.__next__()
x_val, y_val= val_img_gen.__next__()

In [30]:
print(y.size)
print(x_val.size)
print("max value in image dataset is: ", x_val.max())

786432
786432
max value in image dataset is:  0.97647065


In [31]:
#Define the model metrcis and load model. 

num_train_imgs = len(os.listdir('/gdrive/MyDrive/TESI/prova/data/train_images/images'))
num_val_images = len(os.listdir('/gdrive/MyDrive/TESI/prova/data/val_images/images'))
steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size

IMG_HEIGHT = x.shape[1]
IMG_WIDTH  = x.shape[2]
IMG_CHANNELS = x.shape[3]
input_shape = (3,IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
print(input_shape)

(3, 512, 512, 1)


In [32]:
import tensorflow as tf

In [33]:
time_steps = 3
img_height = 512
img_width = 512
channels = 1

In [34]:
num_train_imgs = len(os.listdir('/gdrive/MyDrive/TESI/prova/data/train_images/images'))
num_val_images = len(os.listdir('/gdrive/MyDrive/TESI/prova/data/val_images/images'))
steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size

In [35]:
print(steps_per_epoch)

128


In [36]:
from tensorflow.keras.layers import Conv2D, TimeDistributed,Dropout,Input, Dense,\
    BatchNormalization, GRU, Layer, Flatten,MaxPooling2D, concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from keras import layers
from keras import models

In [37]:
input_l = layers.Input(shape=(time_steps,img_height, img_width,channels))
x =  (layers.TimeDistributed(layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1) ))) (input_l)
conv2 = layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv2)
x = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
conv5 = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv5)
x = layers.TimeDistributed( layers.Conv2D( 256, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
conv8 = layers.TimeDistributed( layers.Conv2D( 256, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
x=layers.TimeDistributed(layers.MaxPooling2D(pool_size=(2,2)))(conv8)
x=layers.Bidirectional(layers.ConvLSTM2D(256,kernel_size=(3,3),padding='same',strides=(1,1),return_sequences=True))(x)
up1 = layers.TimeDistributed( layers.Conv2DTranspose( 512,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat1 = layers.concatenate([up1, conv8])
x = layers.TimeDistributed( layers.Conv2D( 256, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (concat1)
x = layers.TimeDistributed( layers.Conv2D( 256, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
up2 = layers.TimeDistributed( layers.Conv2DTranspose( 256,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat2 = layers.concatenate([up2, conv5])
x = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (concat2)
x = layers.TimeDistributed( layers.Conv2D( 128, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (x)
up3 = layers.TimeDistributed( layers.Conv2DTranspose( 128,kernel_size=(3,3),padding='same',strides=(2,2)))(x)
concat3 = layers.concatenate([up3, conv2])
x = layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(3, 3),padding='same',strides=(1,1) ) ) (concat3)
x=layers.Bidirectional(layers.ConvLSTM2D(32,kernel_size=(3,3),padding='same',strides=(1,1),return_sequences=True))(x)
out = tf.reshape(x, (-1, 1, 512,512, 64))
#out= layers.TimeDistributed( layers.Conv2D( 64, kernel_size=(1, 1),padding='same',strides=(1,1) ) ) (out)


model = models.Model(inputs=input_l, outputs=out)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 3, 512, 512  0           []                               
                                , 1)]                                                             
                                                                                                  
 time_distributed_17 (TimeDistr  (None, 3, 512, 512,  640        ['input_2[0][0]']                
 ibuted)                         64)                                                              
                                                                                                  
 time_distributed_18 (TimeDistr  (None, 3, 512, 512,  36928      ['time_distributed_17[0][0]']    
 ibuted)                         64)                                                        

In [79]:
def dice_coeff(pred, target):
    smooth = 1.
    num = pred.size(0)
    m1 = pred.view(num, -1)  # Flatten
    m2 = target.view(num, -1)  # Flatten
    intersection = (m1 * m2).sum()

    return (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)
def dice_coef_loss(pred, target):
    return 1-dice_coeff(pred, target)

In [68]:
#no
from keras import backend as K
def dice_coef(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection) / (K.sum(K.square(y_true),axis=-1) + K.sum(K.square(y_pred),axis=-1) + epsilon)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)




In [86]:
from keras import backend as K
def dice_coef(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection) / (K.sum(K.square(y_true),axis=-1) + K.sum(K.square(y_pred),axis=-1) + epsilon)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [87]:
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Recall, Precision

In [88]:

model.compile(optimizer=Adam(1e-4), loss=dice_coef_loss, metrics=dice_coef)


In [ ]:
history=model.fit(train_img_gen,
          steps_per_epoch=steps_per_epoch,
          epochs=10,
          verbose=1,
          validation_data=val_img_gen,
    
          validation_steps=val_steps_per_epoch)

Epoch 1/10
128/128 [==============================] - 271s 2s/step - loss: -3.5614 - dice_coef: 4.5614 - val_loss: -4.0126 - val_dice_coef: 5.0126
Epoch 2/10
128/128 [==============================] - 207s 2s/step - loss: -4.0051 - dice_coef: 5.0051 - val_loss: -4.0117 - val_dice_coef: 5.0117
Epoch 3/10
128/128 [==============================] - 208s 2s/step - loss: -4.0156 - dice_coef: 5.0156 - val_loss: -4.0188 - val_dice_coef: 5.0188
Epoch 4/10
128/128 [==============================] - 207s 2s/step - loss: -4.0150 - dice_coef: 5.0150 - val_loss: -4.0201 - val_dice_coef: 5.0201
Epoch 5/10
128/128 [==============================] - 207s 2s/step - loss: -4.0132 - dice_coef: 5.0132 - val_loss: -4.0198 - val_dice_coef: 5.0198
Epoch 6/10
128/128 [==============================] - 207s 2s/step - loss: -4.0187 - dice_coef: 5.0187 - val_loss: -4.0225 - val_dice_coef: 5.0225
Epoch 7/10
128/128 [==============================] - 207s 2s/step - loss: -4.0177 - dice_coef: 5.0177 - val_loss: -4.

In [ ]:

model.save('/gdrive/MyDrive/TESI/sensor3Dfunzionante.hdf5')